# 🏠 DeepRoof-2026: Multi-Task Training Notebook

Welcome to the official training environment for the **DeepRoof-2026 AI Roof Layout Engine**. 

This notebook allows you to:
1. **Visualize** the OmniCity dataset labels (Instance Masks + Surface Normals).
2. **Configure** training parameters for either **Scratch Training** or **Fine-Tuning**.
3. **Launch** the high-performance training loop optimized for A100 GPUs.
4. **Evaluate** and visualize model predictions on new satellite imagery.

In [ ]:
import os
import sys
import subprocess
import torch
from pathlib import Path

# --- 🛠 STEP 1: SOLVE PATHS & VENV ---
project_root = str(Path(os.getcwd()).parent)
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"✅ Added {project_root} to sys.path")

venv_site = os.path.join(project_root, ".venv/lib/python3.11/site-packages")
if os.path.exists(venv_site):
    sys.path.insert(1, venv_site)
    print(f"🐍 Using venv at: {venv_site}")

# --- 📦 STEP 2: VERSION CONFLICT RESOLUTION (MMCV/MMSEG) ---
def check_and_fix_mmcv():
    print("🔍 Checking OpenMMLab Compatibility...")
    try:
        import mmcv
        from mmengine.utils import digit_version
        print(f"🔹 MMCV Version Found: {mmcv.__version__}")
        
        # mmseg often caps mmcv at < 2.2.0. If we have 2.2.0+, we downgrade.
        if digit_version(mmcv.__version__) >= digit_version("2.2.0"):
            print("⚠️ MMCV 2.2.0+ detected. Downgrading to 2.1.0 for mmseg compatibility...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "mmcv==2.1.0", "-f", "https://download.openmmlab.com/mmcv/dist/cu121/torch2.1/index.html"])
            print("✅ MMCV Downgraded. Please RESTART KERNEL after this cell finishes.")
    except ImportError:
        print("📦 MMCV missing. Installing compatible version...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "openmim"])
        subprocess.check_call([sys.executable, "-m", "mim", "install", "mmcv>=2.0.0rc4,<2.2.0"])

def install_packages():
    check_and_fix_mmcv()
    
    dependencies = ["mmengine", "mmsegmentation>=1.0.0", "rasterio", "geopandas", "albumentations"]
    for pkg in dependencies:
        pkg_name = pkg.split(">=")[0]
        try:
            __import__(pkg_name.replace("mmsegmentation", "mmseg"))
        except ImportError:
            print(f"📦 Installing {pkg}...")
            if "mmsegmentation" in pkg:
                subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "openmim"])
                subprocess.check_call([sys.executable, "-m", "mim", "install", pkg])
            else:
                subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

install_packages()

from mmengine.config import Config
from mmengine.runner import Runner

print(f"\n🚀 CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"💻 Device: {torch.cuda.get_device_name(0)}")

## 📂 1. Dataset Preview

Before training, let's look at what our model will see. We combine **Satellite View 1** images with **Instance Masks** (segmentation) and **Surface Normals** (geometry).

In [ ]:
def preview_dataset(data_root, num_samples=3):
    import matplotlib.pyplot as plt
    import numpy as np
    import cv2
    
    data_path = Path(data_root)
    if not data_path.is_absolute():
        data_path = Path(project_root) / data_path
        
    train_file = data_path / 'train.txt'
    if not train_file.exists():
        print(f"❌ Could find train.txt at {train_file}. Run prepare_omnicity_v2_final.py first!")
        return
        
    with open(train_file, 'r') as f:
        sample_ids = [line.strip() for line in f.readlines()[:num_samples]]
    
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, 5 * num_samples))
    for i, sid in enumerate(sample_ids):
        img = cv2.cvtColor(cv2.imread(str(data_path / 'images' / (sid + '.jpg'))), cv2.COLOR_BGR2RGB)
        mask = cv2.imread(str(data_path / 'masks' / (sid + '.png')), cv2.IMREAD_UNCHANGED)
        mask_vis = cv2.applyColorMap(((mask % 20) * 20).astype(np.uint8), cv2.COLORMAP_JET)
        axes[i, 0].imshow(img); axes[i, 0].axis('off')
        axes[i, 1].imshow(mask_vis); axes[i, 1].axis('off')
        if (data_path / 'normals' / (sid + '.npy')).exists():
            normals = np.load(str(data_path / 'normals' / (sid + '.npy')))
            axes[i, 2].imshow(((normals + 1) * 127.5).astype(np.uint8))
        axes[i, 2].axis('off')
    plt.tight_layout(); plt.show()

preview_dataset("data/OmniCity", num_samples=2)

## ⚙️ 2. Training Configuration


In [ ]:
MODE = "fine-tune"
CONFIG_FILE = str(Path(project_root) / "configs/deeproof_finetune_swin_L.py")
WORK_DIR = str(Path(project_root) / "work_dirs/swin_l_omnicity_v2")

cfg = Config.fromfile(CONFIG_FILE)
cfg.work_dir = WORK_DIR
cfg.data_root = str(Path(project_root) / "data/OmniCity/")
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.train_cfg.max_iters = 20000

if MODE == "scratch": cfg.load_from = None
print(f"✅ Config Loaded. Mode: {MODE}")

## 🚀 3. Start Training


In [ ]:
runner = Runner.from_cfg(cfg)
runner.train()